In [ ]:
import os
import torch
import pandas as pd
import scanpy as sc
import seaborn as sns
import numpy as np
import squidpy as sq
import matplotlib.pyplot as plt

In [2]:
import SpatialGlue

In [3]:
# Environment configuration. SpatialGlue pacakge can be implemented with either CPU or GPU. GPU acceleration is highly recommend for imporoved efficiency.
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# the location of R, which is necessary for mclust algorithm. Please replace the path below with local R installation path
os.environ['R_HOME'] = 'C:\Program Files\R\R-4.4.0'

In [4]:
# Specify data type
data_type = 'Spatial-epigenome-transcriptome'

# Fix random seed
from SpatialGlue.preprocess import fix_seed
random_seed = 2022
fix_seed(random_seed)

In [ ]:
# read data

adata_omics1 = sc.read_h5ad("./data/spatial ATAC-RNA-seq E13 mouse embryo/s1_adata_rna.h5ad")
adata_omics2 = sc.read_h5ad("./data/spatial ATAC-RNA-seq E13 mouse embryo/s1_adata_atac.h5ad")

adata_omics1.var_names_make_unique()
adata_omics2.var_names_make_unique()

In [ ]:
from SpatialGlue.preprocess import clr_normalize_each_cell, pca, lsi

# RNA
sc.pp.filter_genes(adata_omics1, min_cells=10)
#sc.pp.filter_cells(adata_omics1, min_genes=1)

sc.pp.highly_variable_genes(adata_omics1, flavor="seurat_v3", n_top_genes=3000)
sc.pp.normalize_total(adata_omics1, target_sum=1e4)
sc.pp.log1p(adata_omics1)
sc.pp.scale(adata_omics1)

adata_omics1_high =  adata_omics1[:, adata_omics1.var['highly_variable']]
adata_omics1.obsm['feat'] = pca(adata_omics1_high, n_comps=50)

# ATAC
import episcanpy as epi
adata_omics2 = adata_omics2[adata_omics1.obs_names].copy()
epi.pp.filter_features(adata_omics2, min_cells=int(adata_omics2.shape[0] * 0.03))
epi.pp.filter_features(adata_omics2, min_cells=1) # .obsm['X_lsi'] represents the dimension reduced feature
if 'X_lsi' not in adata_omics2.obsm.keys():
    sc.pp.highly_variable_genes(adata_omics2, flavor="seurat_v3", n_top_genes=3000)
    lsi(adata_omics2, use_highly_variable=False, n_components=51)

adata_omics2.obsm['feat'] = adata_omics2.obsm['X_lsi'].copy()

In [ ]:
from SpatialGlue.preprocess import construct_neighbor_graph
data = construct_neighbor_graph(adata_omics1, adata_omics2, datatype=data_type)

In [ ]:
# define model
from SpatialGlue.SpatialGlue_pyG import Train_SpatialGlue
model = Train_SpatialGlue(data, datatype=data_type, device=device)

# train model
output = model.train()

In [ ]:
adata = adata_omics1.copy()
adata.obsm['emb_latent_omics1'] = output['emb_latent_omics1']
adata.obsm['emb_latent_omics2'] = output['emb_latent_omics2']
adata.obsm['SpatialGlue'] = output['SpatialGlue']
adata.obsm['alpha'] = output['alpha']
adata.obsm['alpha_omics1'] = output['alpha_omics1']
adata.obsm['alpha_omics2'] = output['alpha_omics2']

In [ ]:
# we set 'mclust' as clustering tool by default. Users can also select 'leiden' and 'louvain'
from SpatialGlue.utils import clustering

tool = 'mclust' # mclust, leiden, and louvain
clustering(adata, key='SpatialGlue', add_key='SpatialGlue', n_clusters=10, method=tool, use_pca=True) 


In [ ]:
sc.pl.embedding(adata, basis='spatial', color='SpatialGlue', title='SpatialGlue', s=150, show=False)